# Integration Skills 1: Integrated Numerical Model for the Water Balance of the Forum Ponds

### M.K. van der Molen, E. Peeters,  and R. Dijksma
### Notebook by Antonija Rimac

## Contents

1. Introduction

1. Water balance      

1. Formulating the water balance model
   
1. Realistic model

1. Sensitivity study

1. Finish the procedure


## Chapter 1: Introduction

In *Integration Skills 1 and 2*, you will focus on development of an integrated numerical model to simulate the quantitative and qualitative water balance of one of the Forum ponds on the Wageningen University campus. You will build on the knowledge you collected in Basic Skills Hydrology, Water Quality, Air Quality, Soil Quality and Meteorology. The quantitative Water Balance model build in *Integration Skills 1*, you will use again in *Integration Skills 2* in order to expand it to a water quality model. The final product will also be used in the *PBL phase* of the course.

<p style="background:powderblue;color:black">(Python coding explanation:) > Some parts of the notebook are again marked in (powder)blue. These parts give some more in-depth explanation about coding in Python/Jupyter Notebooks.<br></p>

Note that the "Jupyter notebook" practical consists of so-called code cells and markdowns cells. Markdown cells are used to write text, pose questions, and for you to write your notes. Code cells are used to write parts of a numerical code or program. In each code cell we write a necessary part of a code and we write some comments that start with the mark "#". In the comment, we explain (again) what we calculate or plot. To obtain a result of a calculation i.e., a figure (note that most of the calculations we have already done for you), you will need to run a cell. You can do that by clicking on a cell and pressing simultaneously **Shift Enter**. Only then you will be able to see the result of the calculation and only then will the figures appear on your screen. Happy notebooking!

<div align=center>
<figure>
  <img src="DataCamp.png", width="300" height="175">
  
</figure>
</div>

<p style="background:powderblue;color:black">The Integration Course is supported by <a href="https://www.datacamp.com/groups/shared_links/ac22b776f22781dd0fb789d58bcef3e6fd7ac3931763d843264b05aa742e975a">DataCamp</a>, an intuitive learning platform for data science and analytics. Learn any time, anywhere and become an expert in R, Python, SQL, and more. DataCamp’s learn-by-doing methodology combines short expert videos and hands-on-the-keyboard exercises to help learners retain knowledge. Subscribe with your @WUR.nl email address and participate in focussed assignments.</p>



___
## Setting up Python modules

We start the tutorial by loading the required Python packages. In the cell below we load the necessary python packages (if some warning signs or requirement signs appear, ignore them).

In [ ]:
# Load necessary python packages. This is to give Python the proper functionality.
#import sys
#!{sys.executable} -m pip install matplotlib pandas cufflinks > /dev/null; # Remove > /dev/null in case of errors.

import os
teacher_dir  = os.getenv('TEACHER_DIR')

import sys
sys.path.append(teacher_dir + '/JHL_data/pictures/')

import nemo
from ipywidgets import interact
import numpy as np
from numpy import log as ln
import matplotlib.pyplot as plt
import pandas as pd
import cufflinks as cf
import os

## Chapter 2: Water balance

The model you will develop here is based on the conservation equation of water in the Forum pond, which is expressed as:

\begin{equation*}
\frac{dV}{dt} = P-E+Q_{k,in}-Q_{k,out}+Q_{l,in}-Q_{l,out}.
\end{equation*}

In the equation, $V$ is the water volume in the pond, $dV/dt$ is the change in water volume in time. This change can be described as the sum of several processes. Those are precipitation $P$, evaporation $E$, vertical in- and outflows $Q_{k,in}$ and $Q_{k,out}$, and lateral in- and outflows $Q_{l,in}$ and $Q_{l,out}$. All terms are given in m$^3$h$^{-1}$. Additional terms may be added to represent e.g., irrigation.

The water volume at any time is the integral of volume changes in the past given as:

\begin{equation*}
V_t = \int_{t_1}^{t_2}\frac{dV}{dt}dt.
\end{equation*}

Since our integral is difficult to solve analytically, we will solve it numerically as:

\begin{equation*}
V_{t+\Delta t} = V_t+\frac{dV}{dt}\times \Delta t.
\end{equation*}

This equation basically formulates that the water volume at time $t+\Delta t$ depends on the volume at time $t$ plus the change in volume during time unit $\Delta t$. This implies a linearization of the differential equation where the time step $\Delta t$ must be chosen sufficiently small for $dV/dt$ to be constant over time unit $\Delta t$.

## Chapter 3 Formulating the water balance model

In the Basic Skills Air Quality you computed NH$_3$ deposition fluxes. The results you obtained for time $t$ did not depend on those at earlier times. This made that the calculations for all time steps could be done at once. In the water balance model the water volume at time $t$ does depend on the water value at time $t - \Delta t$, because e.g., the outflow from the pond depends on the water volume in the pond. Therefore, we need to explicitely integrate in time. Fortunately, such integrations are simple enough to do in a computer model. We only need to set initial conditions and to quantify the terms in the budget equation. 

### Section 3.1 Physical constants and parameters 

Before we set up the initial condition for the model, let's collect necessary physical constants and parameters for the model. 

<ul>
<li><p style="background:powderblue;color:black">Constants we will use in this model are the latent heat of vaporisation `Lv` [J kg$^{-1}$] and the psychrometer constant `gamma` [kPa K$^{-1}$].</li><br>
<li><p style="background:powderblue;color:black">Parameters are settings to the model. They often represent choices you make. As such they are different from physical constants which are fixed in all conditions. In this situation, they are the time step `dt` [h], surface area of the pond A [m$^2$], maximum level of the pond `d_max` [m], deepest level of the pond `d_min` [m] and maximum volume of the pond `V_max` [m$^3$].
</li></ul>

In [ ]:
# Set the value of physical constants.           
Lv     = 2.45e6                # Latent heat of vaporisation [J/kg ]
gamma  = 0.066                 # Psychrometer constant [kPa/K]

# Set the parameters                   
dt     = 1.                    # Time step [h]
A      = 5000.                 # Surface area [m2]
d_max  = 0.                    # Maximum level (height above the surface) [m]
d_min  = -3.                   # Minimum level (depth below the surface)  [m]
V_max  = A * (d_max-d_min)     # Max volume [m3]

### Section 3.2: Initial conditions

The initial condition is the state where we start the model. For simplicity, here we will start with a pond that is filled to the rim. If any more water is added, it will spill over the rim. Thus, the initial volume equals:

\begin{equation*}
V_{ini} = A \times (d_{max} – d_{min}), 
\end{equation*}

where $A$ is the pond’s surface area in m$^2$, and $d_{max}$ and $d_{min}$ are the maximum and minimum depth of the pond in m above sealevel. 

<ul>
<li><p style="background:powderblue;color:black">Assume here that the initial volume `V_ini` [m$^3$] is equal to the maximum value of the pond `V_max`. If the initial value os larger than the maximum value than set it to be equal to the maximum value.</li></ul>

In [ ]:
# Initial conditions
V_ini  = V_max           # The initial Volume [m3] 

# If the initial volume is larger than the maximum volume of the pond, 
# set it to be equal to the maximum value.
if V_ini > V_max :
    V_ini = V_max

### Section 3.3 Tendencies

Tendencies describe how the water volume changes at some point in time due to a specific process. The variables that cause the change are for example precipitation $P$, evaporation $E$ and water flow terms $Q$ in the balance equation. 

In the next cell, we will: 

<ul>
<li><p style="background:powderblue;color:black">Load the meteorological data of global radiation `Rg` [W m$^{-2}$], air temperature `Ta` [degC] and precipitation `P` [mm] measured at Veenkampen station from 2013 to 2021.</li><br>
<li><p style="background:powderblue;color:black">Convert precipitation from mm to m$^3$ h$^{-1}$ using the pond's surface area `A`.</li><br>    
<li><p style="background:powderblue;color:black">For simplicity, we assume (for now) that evaporation $E$ and the $Q$-terms are constant in time.</li><ul>

In [ ]:
# Load the meteorological data. 
# Load precipitation (P; mm), air temperature (Ta; degC) and global radiation (Rg; W m^-2).
teacher_dir        = os.getenv('TEACHER_DIR')
datapath           = teacher_dir + '/JHL_data/IC_data/'

df_meteo           = pd.read_csv(datapath + 'Data_Meteo_Veenkampen_2013-2021.csv', sep=';', 
                                 index_col='date-time', usecols=['date-time', 'Rg', 'Ta', 'P'], 
                                 parse_dates=['date-time'])

# Convert precipitation using the pond's surface area.
df_meteo['P'     ] = df_meteo['P']*A/1000.

# Set evaporation and lateral inflow to have constant values.
df_meteo['E'     ] = 0.5   # m3/hr
df_meteo['Ql_in' ] = 0.1   # m3/hr 
df_meteo['Qk_in' ] = 0.0   # m3/h 
df_meteo['Qk_out'] = 0.0   # m3/h

### Exercise 1: Change of tendencies in time

In this exercise, you will look at the change of precipitation in time. 

### Question 1.1:

1. Does the Veenkampen meteorological station experience wetter and drier seasons?<br>
2. Do you expect Forum ponds at one moment during the to be filled up to the rim and when?<br>

To answer question 1.1 do the following:

- Plot the precipitation as a function of time in the first cell box below.<br> 

- Write your results in the second cell box below. To answer your questions, you should fill in the sentences below. <br>
`1. The Veenkampen meteorological station (x = 'does/does not') experience wetter and drier seasons.`<br>
`2. I (y = 'do/do not') expect the Forum Ponds to be filled up to the rim at the end of the (z1 = 'winter/spring/summer/autumn').`

In [ ]:
# Plot the time series of precipitation. Here 3-monthly total precipitation is shows. 
# Plot Time [year] on x-axis and Precipitation [m^3/h] on y-axis. 
fig1 = df_meteo['P'].groupby(pd.Grouper(freq='Q')).mean().iplot(asFigure=True, xTitle='Time [year]', 
                                                            yTitle='Precipitation [m\u00B3 h\u207B\u00B9]', width=2) 
fig1.show()

In [ ]:
# Write your answers:
# x: The Veenkampen meteorological station (x = 'does/does not') experience wetter and drier seasons. 
# y, z: I (y = 'do/do not') expect the Forum Ponds to be filled up to the rim at the end of the 
# (z = 'winter/spring/summer/autumn').
x = 'does not'
y = 'do'
z = 'winter'

nemo.check_my_answer_ISa1_1(x, y, z)

### Section 3.4 Integration in time

At this point, you have an initial volume of water in the pond $V_{ini}$, the precipitation $P$ and (constant) evaporation $E$, lateral inflow $Q_{l,in}$, and vertical inflow $Q_{k,in}$. 

In the next cell, we will: 

<ul>
<li><p style="background:powderblue;color:black">Setup the initial conditions for volume `df_meteo['V']` in m$^3$, water volume change df_meteo['dV_dt' ] in m$^3$ h$^{-1}$, and lateral outflow `df_meteo['Ql_out']` in m$^3$ h$^{-1}$ of the pond.</li><br>
<li><p style="background:powderblue;color:black">Finally, we will integrate the model in time by computing the volume of the next time step using the equation introduced as: $V_{t+\Delta t} = V_t + dV/dt \times \Delta t$ </li><ul>

In [ ]:
# Initialisation of arrays.
df_meteo['V']                      = V_ini    # m3   initial volume
df_meteo['dV_dt' ]                 = 0.0      # m3/h initial water volume change in time (tencendy)
df_meteo['Ql_out']                 = 0.       # m3/h initial lateral outflow

# Time integration. Here we loop over time, calculate the outflow tendency, calculate the new volume based on the tendencies
# and start again for the next time step.
nt                                 = len(df_meteo)
for it in range(1,nt):
    it_now                         = df_meteo.index[it  ]
    it_prev                        = df_meteo.index[it-1]
    df_meteo.at[it_now, 'Ql_out']  = max(0., df_meteo.at[it_prev, 'V'] - V_max)
    df_meteo.at[it_now, 'dV_dt']   = df_meteo.at[it_now, 'P'    ] - df_meteo.at[it_now, 'E'     ] \
                                   + df_meteo.at[it_now, 'Qk_in'] - df_meteo.at[it_now, 'Qk_out'] \
                                   + df_meteo.at[it_now, 'Ql_in'] - df_meteo.at[it_now, 'Ql_out']
    df_meteo.at[it_now, 'V']       = df_meteo.at[it_prev, 'V']    + df_meteo.at[it_now, 'dV_dt']*dt 

### Exercise 2: Change of volume in time

In this exercise, after you performed time integration of the water volume in the Forum Ponds, you will look at the change of this volume. 

### Question 2.1: 

1. Do you observe a seasonal cycle in the water volume?<br>
2. How can you explain this based on the simple settings we choose for the evaporation and $Q$-terms?<br>
3. In which winters does the pond fill to the rim and in which other years does it not?<br>
4. When is the pond filled up to the rim in 2021 and why?<br>

To answer question 2.1 do the following: 

- Plot the calculated water volume as a function of time in the first cell box below.<br>

- Write your results in the second cell box below. To answer your questions, you should fill in the sentences below. Note that when you are answering the *second* question in stead of `parameter` you need to write a correct name of the parameter influencing the water volume of Forum Ponds. In the *third* question, instead of `lower year/higher year` you need to write the exact year numbers. Finally, in the *fourth* question, instead of `Month` you need to write a full name of the month, while instead of `letter` you need to add a letter that corresponds to the correct answer.<br>
`1. I (x = 'do/do not') observe a seasonal cycle in the water volume.`<br>
`2. We set up a (y1 = 'varying/constant') value for evaporation and Q-terms. This means that the volume will depend only on the amount of ('y2 = 'parameter', e.g. 'radiation','precipitation','evaporation') in each year.`<br>
`3. The pond is filled to the rim in years: z = year1/year2 year (e.g. z='2014/2015', enter 4 winters).`<br>
`4. During 2021, the pond is filled to the rim in (q1 = 'Month'). This happend due to q2:` <br>
`a: accumulated precipitation`<br>
`b: exterme precipitation during summer`<br>
`c: lower evaporation during that year`.

In [ ]:
# Plot the time series of calculated water volume. 
# Plot Time [year] on x-axis and Volume [m^3] on y-axis. 
# Note that for easier recogniotion we divided the volume by 1000. 
units_volume = '[10\u00b3 m\u00b3]'
fig2 = (df_meteo['V']/1000).iplot(asFigure=True, xTitle='Time [year]', yTitle='Volume '+units_volume, width=2) 
fig2.show()

In [ ]:
# Write your answers:
# x: I (x = 'do/do not') observe a seasonal cycle in the water volume.
# y1: We assigned Evaporation E and the Q-terms a (y1 = 'varying/constant') value.
# y2: This means that the volume will depend only on the amount of 
# ('y2 = 'parameter', e.g. 'radiation','precipitation','evaporation') in each year.
# z: The pond is filled to the rim in the winter of: z = year1/year2 (e.g. z='2014/2015', enter 4 winters).
# q1, q2: During 2021, the pond is filled to the rim in (q1 = 'Month'). 
# This happend due to q2: a: accumulated precipitation, b: exterme precipitation during summer or 
# c: lower evaporation during that year.
x  = 'do not'
y1 = 'constant'
y2 = 'precipitation'
z  = '2014/2015', '2015/2016', '2017/2018', '2019/2020'
q1 = 'August'
q2 = 'a'

nemo.check_my_answer_ISa2_1(x, y1, y2, z, q1, q2)

## Chapter 4 Realistic model

So far, we have formulated and tested a quantitative water balance model with very simple assumptions. And it works! Thus, congratulations, you have made the largest step already! You may realise now that you developed a time explicit water balance model. The model formulations are simple, and it produces a result that may be explained based on the input $P$, $E$ and $Q$ tendency terms. 

Of these tendency terms, only the values of precipitation $P$ were realistic. For simplicity, we have assumed that the evaporation $E$ and the $Q$-terms (all except the lateral outflow) were constant in time, and many of the $Q$-terms were even set to zero. In reality, evaporation varies in time and the lateral inflow and vertical flows depend on the land unit your group is  assigned to. So, the only thing left to do is formulate more realistic expressions for the evaporation and the Q-terms.

In the next steps, we will make evaporation $E$ and $Q$-terms (i.e., $Q_{l,in}$, $Q_{k,in}$ and $Q_{k,out}$) more realistic and specific for the pond whose water volume we are trying to simulate. 

### Section 4.1 Evaporation

The constant evaporation rate is not very realistic, because evaporation rate depends on temperature, vapor pressure deficit and/or the available energy for evaporation. There are different approaches towards realistically estimating the evaporation rates from ponds or vegetation. Here, we will introduce two approaches: the Makkink and gradient-resistance approach.

### Exercise 3: Makkink approach (from Basic Skills Meteorology)

In the Basic Skills Meteorology, you learned how to estimate the evaporation based on the Makkink's approach. First, you should estimate the reference evaporation rate, i.e., the evaporation rate that occurs if the soil has optimal conditions, $E_{ref}$ in kg m$^2$ s$^{-1}$ as:

\begin{equation*}
E_{ref} = 0.65\frac{1}{L_v}\frac{s}{s+\gamma}R_g,
\end{equation*} 

where $R_g$ is the global radiation in W m$^{-2}$, $L_v=2.45\cdot 10^6$ kg J$^{-1}$ is the latent heat of vaporization, $\gamma=0.066$ kPa K$^{-1}$ is the psychrometer constant at 20 degrees, and $s$ is the derivative of the saturation vapour pressure deficit in kPa. The saturation vapor pressure deficit can be written as:

\begin{equation*}
s(T) = e_s(T)\frac{\ln(10)\times a \times b}{(b + T)^2},\\
e_s(T) = e_s(0 ^\circ C)\cdot 10^{\frac{a \times T}{b + T}},
\end{equation*}

with $e_s$(0) = 0.6107 kPa, a = 7.5 and b = 237.3 $^\circ $C, and with $T$ as the air temperature in $^{\circ}$C. 

### Question 3.1:

1. How does using the Makkink's approach change the volume compared to the volume obtained using the simple approach with the constant evaporation?<br>
2. How does the volume change in summer?<br>
3. How does the volume change in the winter?<br>

To answer question 3.1 do the following:

<ul>
<li><p style="background:powderblue;color:black">In the first cell box below, set up the constants and make a calculation of the saturation vapor pressure deficit `df_meteo['es']` and the reference evaporation `df_meteo['E_ref']`.</li><br>
<li><p style="background:powderblue;color:black">Calculate the evaporation rate based on the Makkink approach. To get the evaporation rate we  need to translate the reference evaporation to the evaporation of a specific vegetation or land surface type. We look at the evaporation above water. Thus, it can be obtained as: $E=C_xE_{ref}$, where $C_x$ is a factor that depends on a crop type, or in this case water.</li><br>
<li><p style="background:powderblue;color:black">Initialize the arrays for volume and lateral outflow.</li><br>
<li><p style="background:powderblue;color:black">Setup the initial conditions for lateral outflow, water volume change in time and water volume in the pond.</li><br>
<li><p style="background:powderblue;color:black">Finally, we will integrate the model in time by computing the volume of the next time step using the equation introduced as: $V_{t+\Delta t} = V_t + dV/dt \times \Delta t$ </li><br>
<li><p style="background:powderblue;color:black">In the second box below, plot the result for the evaporation rate based on Makkink's approach (blue line) and compare it with the result obtained when using the constant evaporation (orange line).</li><ul>

- Write your results in the third cell box below. If there is more than one correct answer under y = 'letter', separate each answer by a comma, and write each letter under ''. <br>
`1. Using the Makkink method gives (x = 'more/less') realistic simulation of water volume. This happens because the Makkink method depends on (y = 'letter', multiple answers may be correct, then type e.g. y = 'b','c').`<br>
`    'a' periodic change in the atmospheric temperature`,<br>
`    'b' periodic change in the saturation vapour pressure deficit`,<br>
`    'c' periodic change in the global radiation`.<br>
`2. During summer, the pond is (z = 'more/less') full when we use the Makkink method.`<br>
`3. Appart from (u1 = 'lower number/higher number') winter, all winters show full ponds when using the (u2 = 'simple/Makkink') method.`

In [ ]:
# Set up some constants and make a calculation for the saturation vapor pressure deficit es [kPa]
# the reference evaporation E_ref [kg m^2/s] and the crop factor for water Cx. 
es_0                                  = 0.6107      # kPa
a                                     = 7.5         #
b                                     = 237.3       # degC
Cx                                    = 1.25        # 'Crop' factor for water.
df_meteo['es'   ]                     = es_0*10**((a*df_meteo['Ta']) /(b+df_meteo['Ta']))
df_meteo['s'    ]                     = df_meteo['es']*ln(10)*a*b    /(b+df_meteo['Ta'])**2
df_meteo['E_ref']                     = 0.65*(1/Lv)*(df_meteo['s']/(df_meteo['s']+gamma))*df_meteo['Rg']

# Calculate the evaporation rate based on the Makkink approach (E_mak; m^3 h^-1).
# The units change between the reference evaporation and the actual evaporation rate.
df_meteo['E_mak']                     = Cx*df_meteo['E_ref']*3600.*A/1000.

# Initialise arrays.
df_meteo['V_mak'     ]                = V_ini       # m3   initial water volume in the pond
df_meteo['Ql_out_mak']                = 0.          # m3/h initial lateral outflow
df_meteo['dV_dt_mak' ]                = 0.0         # m3/h initial water volume change in time

# Time integration. 
nt = len(df_meteo)
for it in range(1, nt):
    it_now                            = df_meteo.index[it  ]
    it_prev                           = df_meteo.index[it-1]
    df_meteo.at[it_now, 'Ql_out_mak'] = max(0., df_meteo.at[it_prev, 'V_mak'] - V_max)
    df_meteo.at[it_now, 'dV_dt_mak']  = df_meteo.at[it_now, 'P'    ] - df_meteo.at[it_now, 'E_mak'     ] \
                                      + df_meteo.at[it_now, 'Qk_in'] - df_meteo.at[it_now, 'Qk_out'    ] \
                                      + df_meteo.at[it_now, 'Ql_in'] - df_meteo.at[it_now, 'Ql_out_mak']
    df_meteo.at[it_now, 'V_mak']      = df_meteo.at[it_prev,'V_mak'] + df_meteo.at[it_now, 'dV_dt_mak']*dt 

In [ ]:
# Save the volumes in the same data frame for easier plotting.
df_volume = pd.concat([df_meteo['V'], df_meteo['V_mak'], ], axis=1, sort=False)
df_volume.columns = ['V','V_mak']

# Plot the time series of calculated water volume. 
# Plot Time [year] on x-axis and Volume [m^3] on y-axis. 
# Note that for easier recogniotion we divided the volume by 1000. 
fig3 = (df_volume/1000).iplot(asFigure=True, xTitle='Time [year]', yTitle='Volume [10\u00b3 m\u00b3]', width=2) 
fig3.show()

In [ ]:
# Write your answers.
# x: Using the Makkink method gives (x = 'more/less') realistic simulation of water volume.
# y: This happens because the Makkink method depends on (y = 'letter', multiple answers may be correct, e.g. y = 'b','c' ).
#     'a': periodic change in the atmospheric temperature,
#     'b': periodic change in the saturation vapour pressure deficit,
#     'c': periodic change in the global radiation.
# z: During summer, the pond is (z = 'more/less') full when we use the Makkink method.
# u1, u2: Appart from (u1 = 'lower number/higher number') winter, all winters show full ponds 
# when using the (u2 = 'simple/Makkink') method.
x  = 'more'
y  = 'a','b','c'
z  = 'less'
u1 = '2018/2019', '2020/2021'
u2 = 'Makkink'

nemo.check_my_answer_ISa3_1(x, y, z, u1, u2)

### Section 4.2 Background information: Penman or Gradient-resistance approach 
The information presented here is intended as a background information. The Makkink evaporation method is quite simplistic and aimed at quantifying daily or weekly evaporation. A more precise method is the Penman method, based on the gradient resistance approach (which you used to quantify the NH$_3$ deposition flux in Basic Skills Air Quality). In Penman method, evaporation from a pond is limited by the aerodynamical resistance $r_a$. You may therefore express the evaporation rate as:

\begin{equation*}
E=\rho\frac{q_s(T_{water})-q_a}{r_a},
\end{equation*}

where $q_s$ [g kg$^{-1}$, or \%] is the specific saturation humidity at the pond's water temperature, $q_a$ [g kg$^{-1}$, or \%] is the specific humidity of the air at a reference level, $\rho$ [kg m$^{-3}$] is the air density which we may assume here to be a constant at 1.2 kg m$^{-3}$, while $r_a$ [s m$^{-1}$] is the aerodynamical resistance.

Before we can estimate the new evaporation, we need to calculate the saturation and actual specific humidity as:

\begin{equation*}
q_s = \frac{R_d}{R_v}\frac{e_s(T)}{p}, \\
q_a = RH\cdot q_s, \\
e_s(T) = e_s(0 ^\circ C)\cdot10^{\frac{a \times T_{water}}{b+T_{water}}},
\end{equation*}

where $R_d = 287$ J kg$^{-1}$ K$^{-1}$ and $R_v = 462$ J kg$^{-1}$ K$^{-1}$ are the gas constants for dry air and water vapour, respectively, $p = 101.3$ kPa is the air pressure at the sea level, and $RH$ (-) is the relative humidity, $e_s(0) = 0.6107$ kPa, $a = 7.5$ and $b = 237.3 ^\circ $C  and $T$ is the air/water temperature in $^{\circ}$C. 

In this formulation, you need an expression of the water temperature, which usually lags behind the air temperature.

### Exercise 4: Lateral inflow and vertical flow

So far, we chose the lateral inflow $Q_{l,in}$ as a small enough value to ensure that the pond does not drain over time. However, this value is no more than a wild guess. Vertical flow $Q_{l,in}$ into the ponds maybe estimated according to the methods explained in Basic Skills Hydrology, using data from [Dinoloket](https://www.dinoloket.nl/). For this purpose you will make a geological cross section of the campus area to obtain information about water wearing layers and water resisting layers in the subsoil. Subsequently, you may use [Grondwatertools](https://www.grondwatertools.nl/) to create isohypse maps of the campus. With these tools you can estimate $Q_{k,in}$ and $Q_{k,out}$.

N.B.: you may derive the values for the Forum ponds or for your group's land unit.

### Question 4.1:

1. How does implementing realistic values for Q-terms change the pond's volume? <br>

<ul>
<li><p style="background:powderblue;color:black">Set up the constants and make a calculation of the saturation vapor pressure deficit `df_meteo['es']` and the reference evaporation `df_meteo['E_ref']`.</li><br>
<li><p style="background:powderblue;color:black">Calculate the evaporation rate based on the Makkink approach. To get the evaporation rate we  need to translate the reference evaporation to the evaporation of a specific vegetation or land surface type. We look at the evaporation above water. Thus, it can be obtained as: $E=C_xE_{ref}$, where $C_x$ is a factor that depends on a crop type, or in this case water.</li><br>
<li><p style="background:powderblue;color:black">Add realistic values for the Forum pond or your group's land unit.</li><br>    
<li><p style="background:powderblue;color:black">Initialize the arrays for volume and lateral outflow.</li><br>
<li><p style="background:powderblue;color:black">Setup the initial conditions for lateral outflow, water volume change in time and water volume in the pond.</li><br>
<li><p style="background:powderblue;color:black">Finally, we will integrate the model in time by computing the volume of the next time step using the equation introduced as: $V_{t+\Delta t} = V_t + dV/dt \times \Delta t$ </li><br>
<li><p style="background:powderblue;color:black">In the second box below, plot the result for the water volume based on the new realistic values for the Q-terms (green line), Makkink's approach (blue line) and the result obtained when using the constant evaporation (orange line).</li><ul>    

- Write your results in the third cell box below.<br>
`Adding realistic values for Q-terms changes the result only in the (x = 'first/last') several years of the integration.`<br>
`After 2016 V_real in autumn is on average about (y1 = number) m3 less than V_mak and after 2019 this number increases to more than (y2 = number).`

In [ ]:
# Add realistic values for Ql_in, Qk_in and Qk_out
# For horizontal conductivity: Go to Dinoloket.nl --> Ondergrondmodellen --> Select 'BRO Regis II v2.2. --> 
#   Zoom in to campus --> Select transect tool --> draw a transect. Finish the transect by double clicking
#   on the end point. --> Select 'Toon doorsnede' --> Mouseover the top layer and select 
#   'Horizontale doorlatendheid (kh)' --> Read the horizontal conductivity from the color scale.
# For vertical conductivity: In Dinoloket.nl (same transect): Mouseover the first impermeable layer below your pond -->
#   select 'Verticale doorlatendheid (kv)' --> Read out the vertical conductivity from the color scale.
#   select 'Dikte' --> Read out the thickness of the impermeable layer.
# For isohypses: Go to grondwatertools.nl --> Theme Grondwater --> Grondwaterstanden in beeld --> Isohypsen -->
#   Select WVP1 (Water conducting layer 1) --> Select a (not too recent) date & Select a (not too small) 
#   area including several observation points --> Read h2 (upstream), h1 (downstream), z1 (at the pond) 
#   and deltax from the map; Inspect seasonal variability!
#   Select WVP2 (Water conducting layer 2) --> Select area again --> Read z2 from the map

# ENTER REALISTIC VALUES FOR THE FORUM POND OR YOUR GROUP'S LAND UNIT HERE.
kh                             = 5.0/24.    # m/d --> m/h Horizontal conductivity
kv                             = 3.5e-3/24. # m/d --> m/h Vertical conductivity
Ah                             = 100.0      # m2 Vertical area of the area perpendicular to the horizontal water flow.
h2                             = 10.0       # m  Isohypse first  water conducting layer upstream
h1                             =  6.0       # m  Isohypse first  water conducting layer downstream
z1                             =  7.0       # m  Isohypse first  water conducting layer at the pond
z2                             =  8.0       # m  Isohypse second water conducting layer at the pond
deltax                         = 1200.      # m  Horizontal distance between h1 and h2.
deltaz                         = 10.0       # m  Thickness of impermeable layer (from Dinoloket)

df_meteo['Ql_in_real']         = Ah * kh/24. * (h2 - h1)/deltax   # m3/h. kh/24h to convert m/d to m/h. Add ditch water and/or overland flow.
df_meteo['Qk_in_real' ]        = A  * kv/24. * (z2 - z1)/deltaz   # m3/h Vertical inflow  

# ! ENTER REALISTIC VALUES FOR Qk_out OF YOUR GROUP's LANDSCAPE UNIT HERE, based on the depth of the pond 
# and the conductivity of the layer at the bottom of the pond.
df_meteo['Qk_out_real']        = 0.0         # m3/h Vertical outflow  

# Initialise arrays.
df_meteo['V_real'     ]        = V_ini       # m3   water volume in the pond
df_meteo['Ql_out_real']        = 0.          # m3/h lateral outflow
df_meteo['dV_dt_real' ]        = 0.0         # m3/h water volume change in time


# Time integration. 
nt                                     = len(df_meteo)
for it in range(1, nt):
    it_now                             = df_meteo.index[it  ]
    it_prev                            = df_meteo.index[it-1]
    df_meteo.at[it_now, 'Ql_out_real'] = max(0., df_meteo.at[it_prev, 'V_real'] - V_max)
    df_meteo.at[it_now, 'dV_dt_real']  = df_meteo.at[it_now, 'P'         ] - df_meteo.at[it_now, 'E_mak'      ] \
                                       + df_meteo.at[it_now, 'Qk_in_real'] - df_meteo.at[it_now, 'Qk_out_real'] \
                                       + df_meteo.at[it_now, 'Ql_in_real'] - df_meteo.at[it_now, 'Ql_out_real']
    df_meteo.at[it_now, 'V_real']      = df_meteo.at[it_prev,'V_real'    ] + df_meteo.at[it_now, 'dV_dt_real' ]*dt 

In [ ]:
# Save the volumes in the same data frame for easier plotting.
df_volume = pd.concat([df_meteo['V'], df_meteo['V_mak'], df_meteo['V_real']], axis=1, sort=False)
df_volume.columns = ['V','V_mak','V_real']

# Plot the time series of calculated water volume. 
# Plot Time [year] on x-axis and Volume [m^3] on y-axis. 
# Note that for easier recognition we divided the volume by 1000. 
fig4 = (df_volume/1000).iplot(asFigure=True, xTitle='Time [year]', yTitle='Volume [10\u00b3 m\u00b3]', width=2) 
fig4.show()

In [ ]:
# Write your results:
# x: Adding realistic values for Q-terms changes the result mostly in the (x = first/last) several years of the integration.
# y1, y2: From 2016 V_real in autumn is on average about (y1 = number) m3 less than V_mak, 
# and after 2019 this number increases to more than (y2 = number).
x  = 'last'
y1 = 500
y2 = 1000

nemo.check_my_answer_ISa4_1(x, y1, y2)

## Chapter 5: Sensitivity study

At this point in the practical, it is important to reflect about what the water balance is really worth. So far, 

- you set parameters to define the dimensions of the pond,
- you quantified the different tendencies, expressing how much different processes contribute to the water balance,
- you summed the tendency terms and integrated over time, and
- you showed how the resulting volume as a function of time.

The model results may suggest that the results are very certain and accurate. In practise, the model results are as reliable as the model formulation and the input data. In reality, not all relevant processes may be included correctly in the model description and the values assigned to evaporation, precipitation, inflow and outflow may be uncertain. As a result, this does not make the model result 100% unreliable, just like they are not 100% reliable. It is somewhere in between. In your group's PBL phase report, you will be asked to discuss the reliability of your results. You might ask youselves: "How to address this?"

Some numbers are known with a lot of accuracy, while others are less accurate. Additionally, some numbers are important to the model, while others are less important. Fortunately, the model is very flexible and quick. Thus, we can just do numerical experiments:

- Consider which numbers are the least accurate and may still influence the model resuls much.
- Change the values of these numbers by +10% or +50%, run the model and check how much the model results change. If the model results change less than 10% or 50%, the model is not so sensitive to the process associated to that number. If it changes more than 10% or 50% the model is sensitive to that process.
- In your report, you can now quantify how certain the model results are. Before doing the sensitivity study, you did **not know** how accurate the model results are. Now you can tell that the model results are **accurate within a certain confidence interval**, and you can pinpoint the parameters or processes to which the model results are the most sensitive. This is important information for the readers of your report. You do not have to make the model perfect (you won't be able to do that in just one course), but at least you can recommend to better quantify specific parameters or processes in future research.

### Exercise 5: Sensitivity study

In this exercise, you will perform a sensitivity study with respect to the initial condition of volume ($V_{ini}$) and the lateral inflow ($Q_{l,in}$). Use the code cells below for this purpose.

### Question 5.1:

1. Is the model sensitive to the initial volume?<br>
2. Is the model sensitive to the lateral inflow?<br> 
3. Do you think you could make the inflow the dominant process by changing it to a certain level?<br>
4. What is the most sensitive parameter in the model? 

To answer question 5.1 do the following:

- Change the initial volume in the cell box below (multiply by 0.90) and make a plot of your water volume.<br>


- Change the lateral inflow in the cell box below and make a plot of your water volume.<br>

- <b>Play with the model</b>: does it matter if you change several things at the same time? Or is it better to change them one by one? How important are other model parameters and variables? Try to identify the ones to which the model is sensitive and the ones that are not sensitive.<br>

- Write your answers in the third cell box below. Write your answers by filling in the sentences:<br>
`1. The model (x1 = 'is/is not') sensitive to changes in the initial volume. It takes about (x2 = number) years for the pond to fill up in winter with water when we decrease the initial volume by 10%.`<br>
`2. The model (y = 'does/does not') show sensitivity to changes in the lateral inflow. `<br>
`3. We could make the lateral inflow the dominant process by increasing it up to (z = number) m3/h.`<br>
`4. The most sensitive parameter in our model is (u = 'parameter').`

In [ ]:
# Find the sensitivity to Ql,in, Qk,in, Qk,out and initial volume

# Change the values for Qlin, Qk_in and Qk_out
df_meteo['Ql_in_sens' ]                = 1.0/24.     # m3/h Lateral inflow (orig 1.0)   
df_meteo['Qk_in_sens' ]                = 2.0/24.     # m3/h Vertical inflow   
df_meteo['Qk_out_sens']                = 1.0/24.     # m3/h Vertical outflow 
df_meteo['V_sens'     ]                = 1.0 * V_ini # m3   change the initial value by multplying with a factor * 0.90

# Initialise arrays
df_meteo['Ql_out_sens']                = 0.          # m3/h lateral outflow
df_meteo['dV_dt_sens' ]                = 0.0         # m3/h water volume change in time

# Time integration. 
nt                                     = len(df_meteo)
for it in range(1, nt):
    it_now                             = df_meteo.index[it  ]
    it_prev                            = df_meteo.index[it-1]
    df_meteo.at[it_now, 'Ql_out_sens'] = max(0., df_meteo.at[it_prev, 'V_sens'] - V_max)
    df_meteo.at[it_now, 'dV_dt_sens']  = df_meteo.at[it_now, 'P'         ] - df_meteo.at[it_now, 'E_mak'      ] \
                                       + df_meteo.at[it_now, 'Qk_in_sens'] - df_meteo.at[it_now, 'Qk_out_sens'] \
                                       + df_meteo.at[it_now, 'Ql_in_sens'] - df_meteo.at[it_now, 'Ql_out_sens']
    df_meteo.at[it_now, 'V_sens']      = df_meteo.at[it_prev,'V_sens'    ] + df_meteo.at[it_now, 'dV_dt_sens' ]*dt 

In [ ]:
# Save the volumes in the same data frame for easier plotting.
df_volume = pd.concat([df_meteo['V'], df_meteo['V_mak'], df_meteo['V_real'], df_meteo['V_sens']], axis=1, sort=False)
df_volume.columns = ['V','V_mak','V_real','V_sens']

# Plot the time series of calculated water volume. 
# Plot Time [year] on x-axis and Volume [m^3] on y-axis. 
# Note that for easier recogniotion we divided the volume by 1000. 
fig5 = (df_volume/1000).iplot(asFigure=True, xTitle='Time [year]', yTitle='Volume [10\u00b3 m\u00b3]', width=2) 
fig5.show()

In [ ]:
# Write your answers:
# x1: The model (x1 = 'is/is not') sensitive to changes in the initial volume. 
# x2: It takes about (x2 = number) years for the pond to fill up in winter when we decrease the initial volume by 10%. 
# y: The model (y = 'does/does not') show sensitivity to changes in the lateral inflow.
# z: We could make the lateral inflow (Q_l,in) the dominant process by increasing it up to (z = number) m3/24hr.
# u: The most sensitive parameters in our model are (u = 'parameter1','parameter2').
x1 = 'is'
x2 = 2
y  = 'does not'
z  = 3
u  = 'vertical inflow'

nemo.check_my_answer_ISa5_1(x1, x2, y, z, u)


Before you finish, play around with the model. Try to identify important processes, parameters and variables. Try to quantify the sensitivity of the model to those. Can you implement alternative processes, like irrigation? This will help you a lot during the PBL phase of the course.


## Chapter 6: Finish the procedure

To save your output and close the notebook, please run the next cell and follow the steps below.

In [ ]:
df_meteo.to_csv('Data_Output_IntegrationSkills_1.csv', sep=';')

Now, to limit the data storage please do the following:

go to KERNEL --> RESTART and CLEAR OUTPUT. This removes the interactive graphs from your notebook, but leaves your answers intact.
go to FILE --> SAVE and CHECKPOINT. This saves your answers in the notebook
go to FILE --> CLOSE and HALT, to shutdown the notebook. You can always come back later and restart.

In [ ]:
# Check out which data you have written to that file.
df_meteo.keys()